In [1]:
# Run the controller
import torch
import numpy as np
from utils import dtype, npdtype, load_param, get_device
from controllers.mppi_base import ControllerBase
from costs.static import Static
from models.auv_torch import AUVFossen
from observers.observer_base import ObserverBase

/home/pierre/anaconda3/envs/pypose/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_config_file = "../../config/models/rexrov2.default.yaml"
cost_config_file = "../../config/tasks/static_cost.torch.yaml"
cont_config_file = "../../config/controller/rexrov2.torch.yaml"

In [3]:
k = 2000
tau = 50
lam = 0.25
gamma = 0.1
upsilon = 1.5

dt = 0.1

model_conf = load_param(model_config_file)
cost_conf = load_param(cost_config_file)
cont_conf = load_param(cont_config_file)

Q = np.array(cost_conf["Q"], dtype=npdtype)
sigma = np.array(cont_conf["noise"], dtype=npdtype)
goal = np.array(cost_conf["goal"], dtype=npdtype)[..., None]


In [4]:
# GPU execution
device = get_device(gpu=0)
print(device)

model = AUVFossen(model_conf).to(device)
cost = Static(lam=lam,
              gamma=gamma,
              upsilon=upsilon,
              sigma=sigma,
              goal=goal,
              Q=Q).to(device)
obs = ObserverBase(log=False)
cont = ControllerBase(model=model,
                      cost=cost,
                      observer=obs,
                      k=k,
                      tau=tau,
                      lam=lam,
                      upsilon=upsilon,
                      sigma=sigma).to(device)

state = torch.Tensor(np.zeros((3+4+3+3, 1), dtype=npdtype)).to(device)
state[6] = 1.

for i in range(20):
    act = cont(state)
    next_state = model(state[None, ...], act[None, ...])[0]
    state = next_state
timeDict = cont.timeDict

print("Total:", timeDict["total"]/timeDict["calls"])
print("Rollout:", timeDict["rollout"]/timeDict["calls"])
print("Model:", timeDict["model"]/timeDict["calls"])
print("Cost:", timeDict["cost"]/timeDict["calls"])
print("Update:", timeDict["update"]/timeDict["calls"])


True
cuda:0
Total: 1.7197572223999942
Rollout: 1.4353313619000574
Model: 1.3861522053004365
Cost: 0.041875129099889816
Update: 0.00041363500004081287


In [5]:
# CPU execution
device = get_device(cpu=True)
print(device)

model = AUVFossen(model_conf).to(device)
cost = Static(lam=lam,
              gamma=gamma,
              upsilon=upsilon,
              sigma=sigma,
              goal=goal,
              Q=Q).to(device)
obs = ObserverBase(log=False)
cont = ControllerBase(model=model,
                      cost=cost,
                      observer=obs,
                      k=k,
                      tau=tau,
                      lam=lam,
                      upsilon=upsilon,
                      sigma=sigma).to(device)

state = torch.Tensor(np.zeros((3+4+3+3, 1), dtype=npdtype)).to(device)
state[6] = 1.

for i in range(20):
    act = cont(state)
    next_state = model(state[None, ...], act[None, ...])[0]
    state = next_state
timeDict = cont.timeDict

print("Total:", timeDict["total"]/timeDict["calls"])
print("Rollout:", timeDict["rollout"]/timeDict["calls"])
print("Model:", timeDict["model"]/timeDict["calls"])
print("Cost:", timeDict["cost"]/timeDict["calls"])
print("Update:", timeDict["update"]/timeDict["calls"])


cpu
Total: 0.7391295651001201
Rollout: 0.7282633906000683
Model: 0.6831583773001058
Cost: 0.03734689234970574
Update: 0.000721538049992887
